This exmaple will show you how to interpret anomalies in univariate *time-series* samples.

Here we use a HDFS log anomaly detector, named *Deeplog* (CCS'17).


# Prepare the Deeplog model

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Train Deeplog anomaly detection model
import numpy as np
import torch
from deeplog import train_deeplog, test_deeplog
train_data = np.load('./data/train_data.npz')
train_normal_seq = train_data['train_normal_seq']
train_normal_label = train_data['train_normal_label']
model = train_deeplog(train_normal_seq, train_normal_label)
torch.save(model, './save/LSTM_onehot.pth.tar')


Epoch [1/300], train_loss: 3.0241
Epoch [2/300], train_loss: 1.9041
Epoch [3/300], train_loss: 1.7699
Epoch [4/300], train_loss: 1.6208
Epoch [5/300], train_loss: 1.3700
Epoch [6/300], train_loss: 1.0587
Epoch [7/300], train_loss: 0.8723
Epoch [8/300], train_loss: 0.7920
Epoch [9/300], train_loss: 0.7411
Epoch [10/300], train_loss: 0.6915
Epoch [11/300], train_loss: 0.6429
Epoch [12/300], train_loss: 0.5949
Epoch [13/300], train_loss: 0.5595
Epoch [14/300], train_loss: 0.5324
Epoch [15/300], train_loss: 0.5076
Epoch [16/300], train_loss: 0.4880
Epoch [17/300], train_loss: 0.4668
Epoch [18/300], train_loss: 0.4475
Epoch [19/300], train_loss: 0.4332
Epoch [20/300], train_loss: 0.4234
Epoch [21/300], train_loss: 0.4153
Epoch [22/300], train_loss: 0.4066
Epoch [23/300], train_loss: 0.3986
Epoch [24/300], train_loss: 0.3950
Epoch [25/300], train_loss: 0.3886
Epoch [26/300], train_loss: 0.3845
Epoch [27/300], train_loss: 0.3797
Epoch [28/300], train_loss: 0.3767
Epoch [29/300], train_loss: 0

Epoch [232/300], train_loss: 0.1953
Epoch [233/300], train_loss: 0.1947
Epoch [234/300], train_loss: 0.1942
Epoch [235/300], train_loss: 0.1936
Epoch [236/300], train_loss: 0.1943
Epoch [237/300], train_loss: 0.1940
Epoch [238/300], train_loss: 0.1942
Epoch [239/300], train_loss: 0.1932
Epoch [240/300], train_loss: 0.1941
Epoch [241/300], train_loss: 0.1936
Epoch [242/300], train_loss: 0.1933
Epoch [243/300], train_loss: 0.1936
Epoch [244/300], train_loss: 0.1942
Epoch [245/300], train_loss: 0.1945
Epoch [246/300], train_loss: 0.1946
Epoch [247/300], train_loss: 0.1933
Epoch [248/300], train_loss: 0.1923
Epoch [249/300], train_loss: 0.1933
Epoch [250/300], train_loss: 0.1926
Epoch [251/300], train_loss: 0.1931
Epoch [252/300], train_loss: 0.1933
Epoch [253/300], train_loss: 0.1924
Epoch [254/300], train_loss: 0.1920
Epoch [255/300], train_loss: 0.1924
Epoch [256/300], train_loss: 0.1934
Epoch [257/300], train_loss: 0.1927
Epoch [258/300], train_loss: 0.1925
Epoch [259/300], train_loss:

In [3]:
# Validate the performance of trained model
test_normal_loader = np.load('./data/test_normal_loader.npy',allow_pickle=True)
test_abnormal_loader = np.load('./data/test_abnormal_loader.npy',allow_pickle=True)
test_deeplog(model, test_normal_loader, test_abnormal_loader)

elapsed_time: 3048.796s
false positive (FP): 432, false negative (FN): 893, Precision: 88.203%, Recall: 78.341%, F1-measure: 82.980%
Finished Predicting


# Interpret your interested anomaly in four steps 

In [6]:
"""Step 1: Load your model"""
from deeplog import LSTM_onehot
import torch
model = torch.load('save/LSTM_onehot.pth.tar')

"""Step 2: Find an anomaly you are interested in"""
import sys
sys.path.append('../../deepaid/')
from utils import deeplogtools_seqformat
abnormal_data = np.load('data/abnormal_data.npy')
idx = 100
seq, label, anomaly_timeseries = deeplogtools_seqformat(model, abnormal_data, num_candidates=9, index=idx)
# print(seq.shape,label.shape)

"""Step 3: Create a DeepAID Interpreter"""
import sys
sys.path.append("../../deepaid/interpreters/")
from timeseries_onehot import UniTimeseriesAID
feature_desc = np.load('data/log_key_meanning.npy') # feature_description
my_interpreter = UniTimeseriesAID(model, feature_desc=feature_desc, class_num=28)

"""Step 4: Interpret your anomaly and show the result"""
interpretation = my_interpreter(seq, label)
my_interpreter.show_table(anomaly_timeseries, interpretation)

Successfully Initialize <Univariate Timeseries Interptreter> for Model <LSTM_onehot>

Visualize Interpretation (Table View)
+------+-------------------------------------+-------+------+-------------------------------------+
| Ano. |               Meaning               | Diff. | Ref. |               Meaning*              |
+------+-------------------------------------+-------+------+-------------------------------------+
|  4   |      Receiving blk* src&dest:*      |       |  4   |      Receiving blk* src&dest:*      |
|  10  |  PktResponder* for blk* terminating |       |  10  |  PktResponder* for blk* terminating |
|  9   |       PktResponder* Exception       |       |  9   |       PktResponder* Exception       |
|  13  |  Exception in receiveBlock for blk* |       |  13  |  Exception in receiveBlock for blk* |
|  6   |   writeBlock* received exception*   |       |  6   |   writeBlock* received exception*   |
|  7   | PktResponder* for blk* Interrupted. |       |  7   | PktResponder* 